In [ ]:
import urllib.request
from bs4 import BeautifulSoup

# This function will go to doxydonkey.blogspot.in and recursively find all the links to the all the posts
# unless no older posts is found
def getAllDoxyDonkeyPosts(url, links):
    response = urllib.request.urlopen(url)
    soup = BeautifulSoup(response)
    for a in soup.findAll('a'):
        try:
            url = a['href']
            title = a['title']
            if title == 'Older Posts':
                print(title, url)
                links.append(url)
                getAllDoxyDonkeyPosts(url, links)
        except:
            title = ""
    return 


blogUrl = "http://doxydonkey.blogspot.in"
links = []
getAllDoxyDonkeyPosts(blogUrl, links)

In [18]:
# Now we need to parse the article texts from each link
# each article is under an <li> tag and each post may have multiple <li> , and they are in div.class="postBody"
def getDoxyDonkeyText(testUrl):
    response = urllib.request.urlopen(testUrl)
    soup = BeautifulSoup(response)
    mydivs = soup.findAll('div', {'class': 'post-body'})
    
    posts = []
    for div in mydivs:
        posts += map(lambda p: p.text.replace('\xa0',' '), div.findAll('li'))
    return posts

In [ ]:
doxyDonkeyPosts =[]

for link in links:
    doxyDonkeyPosts +=  getDoxyDonkeyText(link)
    
doxyDonkeyPosts

In [21]:
# converts text to TF-IDF representation 
from sklearn.feature_extraction.text import TfidfVectorizer

In [22]:
vectorizer = TfidfVectorizer(max_df=0.5, min_df=2, stop_words='english')

In [23]:
# taks a list of documents and returns a two-dim array in which each row represent one document
X = vectorizer.fit_transform(doxyDonkeyPosts)
# X # outputs a matrix of 2804*13219 - where 2804 is the number of articles in our corpus
# 13219 - the number of columns represents the number of distinct words which are presnet in all the articles. 


<2804x13219 sparse matrix of type '<class 'numpy.float64'>'
	with 280830 stored elements in Compressed Sparse Row format>

In [24]:
from sklearn.cluster import KMeans
# init - alghorithm to choose the centroids in such a way so that it takes min iterations to get the a convergence.
# For are other algorithms choices which you can look up in the documentation
# n_init = 
km = KMeans(n_clusters = 3, init = 'k-means++', max_iter = 100, n_init = 1, verbose = True)

In [25]:
# runs the algorithm for our data
km.fit(X)

Initialization complete
Iteration  0, inertia 5234.044
Iteration  1, inertia 2673.358
Iteration  2, inertia 2667.329
Iteration  3, inertia 2666.629
Iteration  4, inertia 2666.489
Iteration  5, inertia 2666.454
Iteration  6, inertia 2666.441
Iteration  7, inertia 2666.438
Iteration  8, inertia 2666.434
Iteration  9, inertia 2666.430
Iteration 10, inertia 2666.427
Converged at iteration 10: center shift 0.000000e+00 within tolerance 7.308517e-09


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
    n_clusters=3, n_init=1, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=True)

In [28]:
# Every document in our list has been assigned a number for the cluster to which it belongs 
# all of that will be stored in the poperty labels_ ofthe km object
import numpy as np
np.unique(km.labels_, return_counts=True)
# Outputs (array([0, 1, 2], dtype=int32), array([1842,  404,  558]))
# Here 0, 1 and 2 are the labbels for each of the cluster since we had mentioned three clusters

(array([0, 1, 2], dtype=int32), array([1842,  404,  558]))

In [ ]:
# now lets articulate what those themes could be based on which the clustering has been done
text={}
# this will be a dictionary where keys will be cluster numbers and the values will be the
# aggregated text across all the articles present in that cluster
for i, cluster in enumerate(km.labels_):
    oneDocument= docyDonkeysPosts[i]
    if cluster not in text.keys():
        text[cluster] = oneDocument
    else:
        text[cluster] += oneDocument


In [ ]:
# Now we find the most frequent words in each cluster
from nltk.tokenize import sent_tokeniz, word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from collections import defaultdict
from string import punctuation
from heapq import nlargest
import nltk

In [ ]:
_stopwords = set(stopwords.words('english') + list(punctuation) + ['“', '”', '\'s', '', '’', 'y/y', 'millions', 'billions', 'year', 'million', 'billion'])


In [ ]:
# Top keywords in each cluster and their counts
